### 불균형 클래스  
### 클래스 불균형에 대한 이해
### 클래스 가중치 사용
### 리샘플링 기법
### 적절한 평가지표
---

#### 불균형 데이터 생성(1:9) 악성 0 : 양성 1

In [49]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X,y = data.data, data.target
np.random.seed(10)


# 악성을 소수 클래스로 생성
import numpy as np

print(f'악성 양성의 오리지널 비율 :np.unique(y, return_counts=True)')        # np.unique() : 넘파이(Numpy)의 고유값(중복 없는 값) 을 찾는 함수/ return_counts=True : 각 고유값이 몇 번씩 등장했는지 개수를 같이 반환하라는 옵션
m_index = np.where(y==0)[0]      # 악성 index 반환 / np.where(조건식) 조건을 만족하는 원소의 인덱스(위치) 를 반환/ np.where()의 반환형은 튜플(tuple) 입니다.즉, (인덱스배열,) 형태로 나와요. 그래서 [0]을 붙여서 진짜 “인덱스 배열”만 꺼내는 겁
b_index = np.where(y==1)[0]      # 양성 index 반환 

# 악성을 일부만, 양성을 더 많이 사용
# 악성의 30%만 사용, 양성은 전체 1.5: 8.5 비율
size_30 = int(len(m_index)*0.2)
seleted_m_index = np.random.choice(m_index,size=size_30,replace=False)
seleted_b_index = b_index

concatenate_selected_index = np.concatenate([seleted_m_index,seleted_b_index]) # 두 개를 하나로 합친다
np.random.shuffle(concatenate_selected_index) # 위에서 합친 리스트를 섞어준다

X_imb = X[concatenate_selected_index]
y_imb = y[concatenate_selected_index]

# 클래스 분포 확인
unique, counts = np.unique(y_imb,return_counts=True)
print('클래스 분포 :')
for label, count in zip(unique,counts):
    percentage = count / len(y_imb) * 100
    print(f'클래스 {label} : {count} 개 ({percentage:.1f}%)')

# 불균형인 상태로 진행
# 스케일링 정규화 StandatdScaler
# LogisticRegrassion
# Pipe
# 평가는 class report


악성 양성의 오리지널 비율 :np.unique(y, return_counts=True)
클래스 분포 :
클래스 0 : 42 개 (10.5%)
클래스 1 : 357 개 (89.5%)


In [50]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 데이터 분류
x_train, x_test, y_train, y_test = train_test_split(X_imb,y_imb, stratify=y_imb,test_size=0.2,random_state=42)

print('1. 기본 모델 (불균형 무시)')
pipe = Pipeline ([
    ('scaler',StandardScaler()),
    ('clf',LogisticRegression(random_state=42,max_iter=1000))
])
pipe.fit(x_train,y_train)
print(classification_report(y_test,pipe.predict(x_test), target_names=['악성(0)', '양성(1)']))

1. 기본 모델 (불균형 무시)
              precision    recall  f1-score   support

       악성(0)       1.00      0.75      0.86         8
       양성(1)       0.97      1.00      0.99        72

    accuracy                           0.97        80
   macro avg       0.99      0.88      0.92        80
weighted avg       0.98      0.97      0.97        80



In [61]:

print('불균형 해결 : 클래스 가중치 사용')
# 기존파이프라인의 clf 이름의 객체의 파라메터를 조정
from copy import deepcopy
pipe_weight = deepcopy(pipe)
pipe_weight.set_params(clf__class_weight = 'balanced')

# pipe_weight = Pipeline([
#     ('scaler',StandardScaler()),
#     ('clf', LogisticRegression(random_state=10,max_iter=1000,class_weight = 'balanced'))
# ])


pipe_weight.fit(x_train,y_train)
print( classification_report(y_test, pipe_weight.predict(x_test)  )   )

print('가중치 계산')
n_samples = len(y_train)
n_classes = 2
class_counts = np.bincount(y_train)
for i in range(n_classes):
    weight = n_samples / (n_classes * class_counts[i])
    print(f'클래스 : {i} : {weight:.3f}')

불균형 해결 : 클래스 가중치 사용
              precision    recall  f1-score   support

           0       0.88      0.88      0.88         8
           1       0.99      0.99      0.99        72

    accuracy                           0.97        80
   macro avg       0.93      0.93      0.93        80
weighted avg       0.97      0.97      0.97        80

가중치 계산
클래스 : 0 : 4.691
클래스 : 1 : 0.560


In [62]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
print('불균형 : RandomFores (균형모드)')
pipe_rf = Pipeline([
    ('scaler',StandardScaler()),
    ('clf',RandomForestClassifier(class_weight = 'balanced',random_state = 42))
])
pipe_rf.fit(x_train,y_train)
y_pred = pipe_rf.predict(x_test)
print(classification_report(y_test,y_pred))

불균형 : RandomFores (균형모드)
              precision    recall  f1-score   support

           0       1.00      0.62      0.77         8
           1       0.96      1.00      0.98        72

    accuracy                           0.96        80
   macro avg       0.98      0.81      0.87        80
weighted avg       0.96      0.96      0.96        80



In [63]:
%pip install imbalanced-learn

  Using cached imbalanced_learn-0.14.0-py3-none-any.whl.metadata (8.8 kB)
Using cached imbalanced_learn-0.14.0-py3-none-any.whl (239 kB)
Note: you may need to restart the kernel to use updated packages.


In [68]:
# 오버, 언더 샘플링  smote, smoteen 사용해서

from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_imb,y_imb)
